# Эксперементальная часть

Импортируем нужные бибилиотеки и создаем датасет

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

from uplift import *

2025-10-26 16:22:20.417547: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-26 16:22:20.557961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761484940.614603    1805 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761484940.630771    1805 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761484940.745062    1805 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
DATASETS = ['CO', 'CP']
all_datasets = {}

# Загрузка
for name in tqdm(DATASETS):
    train_df, vali_df, test_df, num_users, num_items, item_pop = get_dataset(name, path_to_data=".")
    all_datasets[name] = {
        "train": train_df,
        "vali": vali_df,
        "test": test_df,
        "num_users": num_users,
        "num_items": num_items,
        "item_popularity": item_pop
    }


100%|██████████| 2/2 [00:34<00:00, 17.18s/it]


In [3]:
evaluator = Evaluator()
measures = ['CPrec_10', 'CPrec_100', 'CDCG', 'CDCG_100']
propensity_values = []
casual_values = []

In [4]:
base_rng = np.random.default_rng(12345)
seeds = base_rng.integers(low=0, high=10**9, size=5)
seeds # для нескольких запусков модели

array([699215031, 227336022, 788646988, 316758339, 204176893])

# Propensity модели

In [5]:
import torch.nn.functional as F

from scipy.stats import kendalltau
from sklearn.metrics import f1_score
from scipy.stats import entropy

def predict_z(df_train, df_val, epsilon=0.2):
    p_train = df_train['pred'].values
    norm_mean = float(p_train.mean())
    norm_std = float(p_train.std())
    
    p_pred = df_val['pred'].values

    p_norm = (p_pred - norm_mean) / norm_std
    z_pred = (p_norm >= epsilon).astype(int)
    return z_pred

def get_propensity_metrics(df_train, df_val, epsilon=0.2):
    """
    Оценка качества предсказания propensity scores:
    - KLD между истинным p_true и предсказанным p_pred,
    - Kendall’s Tau между ними,
    - F1-score бинарной классификации exposure с порогом epsilon.
    """
    p_pred = df_val['pred'].values
    p_true = df_val['propensity'].values

    z_true = df_val['treated'].values
    z_pred = predict_z(df_train, df_val)
    
    p_pred = np.clip(p_pred, 1e-6, 1 - 1e-6)
    p_true = np.clip(p_true, 1e-6, 1 - 1e-6)

    kld = entropy(p_true, p_pred)
    tau, _ = kendalltau(p_true, p_pred)
    f1 = f1_score(z_true, z_pred)
    
    return {'kld': kld, 'tau': tau, 'f1': f1}

## Эксперемент 4: Сравнение propensity

### RandomBase

In [ ]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for seed in tqdm(seeds, desc=f"Runs on {dataset}", position=1, leave=False):
        model = RandomBase(all_datasets[dataset]['num_users'], all_datasets[dataset]['num_items'])
        model_name = "RandomBase"

        model.fit(all_datasets[dataset]['train'])

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])
        all_datasets[dataset]['train'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['train'])

        value = get_propensity_metrics(df_train=all_datasets[dataset]['train'].rename(columns={f"{model_name}_pred": "pred"}),
                                        df_val=all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), epsilon=0.2)
        value["model"] = model_name
        value["dataset"] = dataset
        propensity_values.append(value)

Dataset:   0%|          | 0/2 [00:00<?, ?it/s]

{'kld': np.float64(0.9388043105934193), 'tau': np.float64(-4.950504606197425e-05), 'f1': 0.25671376334820256} {'kld': np.float64(0.9389020004862865), 'tau': np.float64(-4.950466794449602e-05), 'f1': 0.22717067309400815}


{'kld': np.float64(0.9387870600287442), 'tau': np.float64(5.87220945903375e-05), 'f1': 0.25667898471130174} {'kld': np.float64(0.9388898540169508), 'tau': np.float64(5.871660250145151e-05), 'f1': 0.22712122727225706}


{'kld': np.float64(0.9389031146702748), 'tau': np.float64(2.9589920423224123e-05), 'f1': 0.25664160135318903} {'kld': np.float64(0.9390040791634029), 'tau': np.float64(2.9594298416231692e-05), 'f1': 0.22708004017088007}


{'kld': np.float64(0.9387153215150433), 'tau': np.float64(7.455194184387505e-05), 'f1': 0.2566605327896183} {'kld': np.float64(0.9388156304024016), 'tau': np.float64(7.455516889625732e-05), 'f1': 0.22719794941985044}


...

{'kld': np.float64(1.4348942359913341), 'tau': np.float64(0.00021691150885053327), 'f1': 0.2566004598083176} {'kld': np.float64(1.4350319337030566), 'tau': np.float64(0.00021691145932085812), 'f1': 0.22711835954581422}


{'kld': np.float64(1.4352525374337914), 'tau': np.float64(-0.00014902701387134375), 'f1': 0.25660808357475406} {'kld': np.float64(1.4353932195455803), 'tau': np.float64(-0.00014902692526501702), 'f1': 0.22722141376210706}


{'kld': np.float64(1.4349788972088413), 'tau': np.float64(7.04801105489145e-05), 'f1': 0.2565524051108811} {'kld': np.float64(1.435104386265683), 'tau': np.float64(7.048020867551325e-05), 'f1': 0.2271084216873612}


Dataset: 100%|██████████| 2/2 [04:35<00:00, 137.68s/it]

{'kld': np.float64(1.4343482526790643), 'tau': np.float64(0.0001313931443565275), 'f1': 0.2566406220233451} {'kld': np.float64(1.4344891309701588), 'tau': np.float64(0.0001313927362532879), 'f1': 0.2271644426568644}


### PopularBase

Запускаем только 1 раз так как, тут ничего не зависит от random seed

In [ ]:
for dataset in tqdm(DATASETS):
    model = PopularBase(all_datasets[dataset]['num_users'], all_datasets[dataset]['num_items'])
    model_name = "PopularBase"

    model.fit(all_datasets[dataset]['train'])

    all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])
    all_datasets[dataset]['train'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['train'])

    value = get_propensity_metrics(df_train=all_datasets[dataset]['train'].rename(columns={f"{model_name}_pred": "pred"}),
                                    df_val=all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), epsilon=0.2)
    value["model"] = model_name
    value["dataset"] = dataset
    propensity_values.append(value)

 50%|█████     | 1/2 [00:20<00:20, 20.88s/it]

{'kld': np.float64(0.8702415033803848), 'tau': np.float64(0.24010317065025308), 'f1': 0.028714803511645785} {'kld': np.float64(0.8748721000699091), 'tau': np.float64(0.24010308228543284), 'f1': 0.26932261112792066}


100%|██████████| 2/2 [00:40<00:00, 20.42s/it]

{'kld': np.float64(0.5410552712778826), 'tau': np.float64(0.7863079021412096), 'f1': 0.018896761914428236} {'kld': np.float64(0.5414550794410529), 'tau': np.float64(0.7863079021412096), 'f1': 0.5588533544179842}


### PropCare

In [20]:
for dataset in tqdm(DATASETS, desc="Datasets", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1):
        model = PropCare.load_model(dir_path=f"saved_models/propcare/best_{dataset}", model_name=f"model_{i+1}", device='cuda')
        model_name = "PropCare"

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = model.get_metrics(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}))
        value["model"] = model_name
        value["dataset"] = dataset
        propensity_values.append(value)

Datasets: 100%|██████████| 2/2 [04:22<00:00, 131.17s/it]


### Итоги

In [21]:
summary = (
    pd.DataFrame(propensity_values)
    .groupby(['model', 'dataset'])
    .agg(
        kld_mean=('kld', 'mean'),
        kld_std=('kld', lambda x: np.std(x, ddof=0)),
        tau_mean=('tau', 'mean'),
        tau_std=('tau', lambda x: np.std(x, ddof=0)),
        f1_mean=('f1', 'mean'),
        f1_std=('f1', lambda x: np.std(x, ddof=0)),
    )
    .reset_index()
)
summary

,model,dataset,kld_mean,kld_std,tau_mean,tau_std,f1_mean,f1_std
0,PopularBase,CO,0.874872,0.000000,0.240103,0.000000,0.269323,0.000000
1,PopularBase,CP,0.541455,0.000000,0.786308,0.000000,0.558853,0.000000
2,PropCare,CO,0.905728,0.011356,0.175620,0.005700,0.388406,0.001946
3,PropCare,CP,0.900241,0.024333,0.431493,0.017465,0.547325,0.003251
4,RandomBase,CO,0.938936,0.000090,0.000022,0.000045,0.227134,0.000044
5,RandomBase,CP,1.434919,0.000339,0.000079,0.000123,0.227170,0.000053


## Эксперемент 6: оценка casual на основе DLCE

### Groynd-Truth

In [6]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "DLCE_original"
        model = DLCE.load_model(dir_path=f"saved_models/dlce/best_{dataset}", model_name=f"model_{i+1}", device='cuda')

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [01:51<00:00, 55.76s/it]


### Random

In [7]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "DLCE_random"
        model = DLCE.load_model(dir_path=f"saved_models/dlce/best_random_{dataset}", model_name=f"model_{i+1}", device='cuda')

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [01:52<00:00, 56.16s/it]


### Popular

In [8]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "DLCE_popular"
        model = DLCE.load_model(dir_path=f"saved_models/dlce/best_popular_{dataset}", model_name=f"model_{i+1}", device='cuda')

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [01:52<00:00, 56.38s/it]


### PropCare

In [9]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "DLCE_propcare"
        model = DLCE.load_model(dir_path=f"saved_models/dlce/best_propcare_{dataset}", model_name=f"model_{i+1}", device='cuda')

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [01:50<00:00, 55.09s/it]


## Итого

In [10]:
agg_dict = {
    f"{m}_mean": (m, "mean") for m in measures
} | {
    f"{m}_std": (m, lambda x: np.std(x, ddof=0)) for m in measures
}

summary = (
    pd.DataFrame(casual_values)
    .groupby(['model', 'dataset'])
    .agg(**agg_dict)
)
summary

CPrec_10_mean  CPrec_100_mean  CDCG_mean  \
model         dataset                                             
DLCE_original CO            0.171910        0.077625   6.793327   
              CP            0.088003        0.088506   7.283449   
DLCE_popular  CO            0.021758        0.020282   5.645130   
              CP            0.189684        0.085453   6.840123   
DLCE_propcare CO            0.018787        0.015847   5.622955   
              CP            0.154595        0.084997   6.938097   
DLCE_random   CO            0.004677        0.009848   5.291087   
              CP            0.033625        0.024632   5.297132   

                       CDCG_100_mean  CPrec_10_std  CPrec_100_std  CDCG_std  \
model         dataset                                                         
DLCE_original CO            1.947316      0.001732       0.000402  0.007517   
              CP            1.878666      0.003854       0.001407  0.013677   
DLCE_popular  CO            0.433364      0.001159       0.000307  0.008055   
              CP            2.134262      0.001933       0.000849  0.010562   
DLCE_propcare CO            0.337625      0.003177       0.000639  0.009180   
              CP            2.013790      0.004903       0.000981  0.019130   
DLCE_random   CO            0.190301      0.001285       0.001067  0.011475   
              CP            0.553320      0.001795       0.000601  0.010897   

                       CDCG_100_std  
model         dataset                
DLCE_original CO           0.008403  
              CP           0.029609  
DLCE_popular  CO           0.009565  
              CP           0.016759  
DLCE_propcare CO           0.013266  
              CP           0.028583  
DLCE_random   CO           0.018641  
              CP           0.015807

# Casual модели

In [ ]:
import pickle

with open("all_datasets.pkl", "rb") as f:
    all_datasets = pickle.load(f)


## RandomBase

In [ ]:
casual_values = []
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "RandomBase"

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [03:26<00:00, 103.48s/it]


## PopularBase

In [28]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "PopularBase"

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [00:56<00:00, 28.21s/it]


## DLCE

In [29]:
for dataset in tqdm(DATASETS, desc="Dataset", position=0):
    for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
        model_name = "DLCE_original"
        model = DLCE.load_model(dir_path=f"saved_models/dlce/best_{dataset}", model_name=f"model_{i+1}", device='cuda')

        all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

        value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
        value["model"] = model_name
        value["dataset"] = dataset
        casual_values.append(value)

Dataset: 100%|██████████| 2/2 [01:52<00:00, 56.04s/it]


In [30]:
agg_dict = {
    f"{m}_mean": (m, "mean") for m in measures
} | {
    f"{m}_std": (m, lambda x: np.std(x, ddof=0)) for m in measures
}

summary = (
    pd.DataFrame(casual_values)
    .groupby(['model', 'dataset'])
    .agg(**agg_dict)
)
summary

CPrec_10_mean  CPrec_100_mean  CDCG_mean  \
model         dataset                                             
DLCE_original CO            0.171910        0.077625   6.793327   
              CP            0.088003        0.088506   7.283449   
PopularBase   CO            0.053443        0.023746   5.825522   
              CP            0.054482        0.040598   6.098322   
RandomBase    CO            0.003673        0.004271   5.017716   
              CP            0.004158        0.004353   4.942555   

                       CDCG_100_mean  CPrec_10_std  CPrec_100_std  CDCG_std  \
model         dataset                                                         
DLCE_original CO            1.947316      0.001732       0.000402  0.007517   
              CP            1.878666      0.003854       0.001407  0.013677   
PopularBase   CO            0.582403      0.000000       0.000000  0.000000   
              CP            0.868866      0.000000       0.000000  0.000000   
RandomBase    CO            0.087671      0.000439       0.000250  0.001886   
              CP            0.090615      0.000744       0.000242  0.002113   

                       CDCG_100_std  
model         dataset                
DLCE_original CO           0.008403  
              CP           0.029609  
PopularBase   CO           0.000000  
              CP           0.000000  
RandomBase    CO           0.005336  
              CP           0.005025

# Различные моды DLCE

In [5]:
for dlto_mode in [True, False]:
    for tau_mode in ['ips', 'cips', 'naive']:
        for dataset in tqdm(DATASETS, desc="Dataset", position=0):
            for i in tqdm(range(len(seeds)), desc=f"Runs on {dataset}", position=1, leave=False):
                model_name = f"DLCE_{dlto_mode}_{tau_mode}"
                model = DLCE.load_model(dir_path=f"saved_models/dlce/modes/dlto{dlto_mode}_{tau_mode}_{dataset}", model_name=f"model_{i+1}", device='cuda')

                all_datasets[dataset]['test'][f"{model_name}_pred"] = model.predict(all_datasets[dataset]['test'])

                value = evaluator.evaluate(all_datasets[dataset]['test'].rename(columns={f"{model_name}_pred": "pred"}), measures=measures)
                value["model"] = model_name
                value["dataset"] = dataset
                casual_values.append(value)

Dataset: 100%|██████████| 2/2 [02:00<00:00, 60.16s/it]


In [6]:
agg_dict = {
    f"{m}_mean": (m, "mean") for m in measures
} | {
    f"{m}_std": (m, lambda x: np.std(x, ddof=0)) for m in measures
}

summary = (
    pd.DataFrame(casual_values)
    .groupby(['model', 'dataset'])
    .agg(**agg_dict)
)
summary

CPrec_10_mean  CPrec_100_mean  CDCG_mean  \
model            dataset                                             
DLCE_False_cips  CO            0.171728        0.077582   6.792906   
                 CP            0.088367        0.088550   7.284219   
DLCE_False_ips   CO            0.171910        0.077625   6.793327   
                 CP            0.088003        0.088506   7.283449   
DLCE_False_naive CO            0.057514        0.016896   5.489560   
                 CP            0.109589        0.074968   7.097955   
DLCE_True_cips   CO            0.148298        0.070106   6.745000   
                 CP            0.103898        0.097103   7.294305   
DLCE_True_ips    CO            0.147934        0.070192   6.743509   
                 CP            0.103898        0.097103   7.294305   
DLCE_True_naive  CO            0.053443        0.025827   5.878835   
                 CP            0.090585        0.064259   6.786157   

                          CDCG_100_mean  CPrec_10_std  CPrec_100_std  \
model            dataset                                               
DLCE_False_cips  CO            1.946146  1.884684e-03       0.000441   
                 CP            1.879309  3.233122e-03       0.001313   
DLCE_False_ips   CO            1.947316  1.732265e-03       0.000402   
                 CP            1.878666  3.853558e-03       0.001407   
DLCE_False_naive CO            0.464600  6.938894e-18       0.000130   
                 CP            1.681115  6.391744e-03       0.000869   
DLCE_True_cips   CO            1.728709  2.634570e-03       0.000555   
                 CP            2.098142  4.790822e-03       0.002264   
DLCE_True_ips    CO            1.728789  2.980123e-03       0.000464   
                 CP            2.098142  4.790822e-03       0.002264   
DLCE_True_naive  CO            0.660594  0.000000e+00       0.001079   
                 CP            1.447456  7.923525e-03       0.001130   

                          CDCG_std  CDCG_100_std  
model            dataset                          
DLCE_False_cips  CO       0.007859      0.008793  
                 CP       0.014890      0.029840  
DLCE_False_ips   CO       0.007517      0.008403  
                 CP       0.013677      0.029609  
DLCE_False_naive CO       0.002463      0.002915  
                 CP       0.020650      0.029546  
DLCE_True_cips   CO       0.009815      0.015481  
                 CP       0.009118      0.032013  
DLCE_True_ips    CO       0.010707      0.014443  
                 CP       0.009118      0.032013  
DLCE_True_naive  CO       0.007962      0.019477  
                 CP       0.020451      0.037564

## Анализ покупок для каждой propensity-модели

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

def guess_items_map_path(dataset_name: str, base_dir="data/preprocessed",
                         min_user=10, min_item=10, min_user_tr=1, min_item_tr=1,
                         rec_type="mailer"):
    """
    Пытаемся угадать, где лежит items_map.csv.
    Для категорийного уровня (CP/CO) это dunn_cat_..., для продуктового — dunn_...
    """
    cat_dir = os.path.join(base_dir, f"dunn_cat_{rec_type}_{min_user}_{min_item}_{min_user_tr}_{min_item_tr}")
    prod_dir = os.path.join(base_dir, f"dunn_{rec_type}_{min_user}_{min_item}_{min_user_tr}_{min_item_tr}")

    candidates = [
        os.path.join(cat_dir, "items_map.csv"),
        os.path.join(prod_dir, "items_map.csv"),
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Не найден items_map.csv ни в {candidates}")

def attach_item_names(split_df: pd.DataFrame, item_map: pd.DataFrame) -> pd.DataFrame:
    """
    Добавляет к df колонки из item_map по idx_item.
    Возвращает НОВЫЙ датафрейм (оригинал не мутируется).
    """
    cols_to_keep = [c for c in item_map.columns if c in
                    ["idx_item", "item_id", "SUB_COMMODITY_DESC", "COMMODITY_DESC",
                     "BRAND", "MANUFACTURER", "DEPARTMENT", "CURR_SIZE_OF_PRODUCT"]]
    m = item_map[cols_to_keep].copy()

    out = split_df.merge(m, on="idx_item", how="left")

    # Удобное читабельное поле: для категорий берём SUB_COMMODITY_DESC,
    # при её отсутствии — COMMODITY_DESC; (на продуктовом уровне тоже ок — оба поля есть)
    if "SUB_COMMODITY_DESC" in out.columns and "COMMODITY_DESC" in out.columns:
        out["item_name"] = out["SUB_COMMODITY_DESC"].fillna(out["COMMODITY_DESC"])
    elif "COMMODITY_DESC" in out.columns:
        out["item_name"] = out["COMMODITY_DESC"]
    else:
        # самый минимум — вернуть item_id как имя
        out["item_name"] = out.get("item_id", out["idx_item"].astype(str))

    return out

# === основной цикл загрузки и обогащения ===
DATASETS = ["CO", "CP"]
all_datasets = {}

for name in tqdm(DATASETS):
    train_df, vali_df, test_df, num_users, num_items, item_pop = get_dataset(name, path_to_data=".")

    items_map_path = guess_items_map_path(name)            # можно явно подать base_dir/параметры, если отличались
    item_map = pd.read_csv(items_map_path)

    # приведём тип индекса к int (на всякий случай)
    if item_map["idx_item"].dtype != "int64":
        item_map["idx_item"] = item_map["idx_item"].astype("int64")

    train_named = attach_item_names(train_df, item_map)
    vali_named  = attach_item_names(vali_df,  item_map)
    test_named  = attach_item_names(test_df,  item_map)

    all_datasets[name] = {
        "train": train_named,
        "vali": vali_named,
        "test": test_named,
        "num_users": num_users,
        "num_items": num_items,
        "item_popularity": item_pop,
        "item_map": item_map,   # чтобы было под рукой
    }

# Пример: посмотреть топ-10 популярных по названию в CP
(all_datasets["CP"]["train"]
 .groupby("item_name")["idx_item"]
 .count()
 .sort_values(ascending=False)
 .head(10))


100%|██████████| 2/2 [00:40<00:00, 20.16s/it]


item_name
SNACK CAKE - MULTI PACK           46210
ANTI-ACIDS                        41225
CONTINUITY: FRAMES                36481
DIET CNTRL BARS NUTRITIONAL       34985
CANDY BARS (SINGLES)(INCLUDING    34078
SEASONAL                          33747
FRESH                             31949
FRZN BREADED PREPARED CHICK       26998
STIR FRY/STRIPS/FAJITAS           25245
BREAD:ITALIAN/FRENCH              24863
Name: idx_item, dtype: int64

In [44]:
model_gt  = DLCE.load_model(dir_path=f"saved_models/dlce/best_CP", model_name=f"model_{1}", device='cuda')
model_pop  = DLCE.load_model(dir_path=f"saved_models/dlce/best_propcare_CP", model_name=f"model_{1}", device='cuda')
model_pc = DLCE.load_model(dir_path=f"saved_models/dlce/best_popular_CP", model_name=f"model_{1}", device='cuda')


In [ ]:
import numpy as np
import pandas as pd

USER_ID = 238
TOPK = 10

# 1) берём тест и мапу названий
test = all_datasets['CP']['test'].copy()
item_map = all_datasets['CP']['item_map'].copy()

# 2) считаем скоры трёх моделей
test_user = test[test['idx_user'] == USER_ID].copy()


test_user['score_gt']  = model_gt.predict(test_user)
test_user['score_pop'] = model_pop.predict(test_user)
test_user['score_pc']  = model_pc.predict(test_user)

test_user = test_user[test_user['idx_time'] == 9]

# 3) агрегируем по товару, чтобы корректно рисовать иконки
agg = (test_user
       .groupby(['idx_item','item_name'], as_index=False)
       .agg(outcome_any=('outcome', lambda x: int(np.max(x) > 0)),
            treated_any=('treated', 'max'),
            causal_sign=('causal_effect', 'max') ,  # -1,0,1
            score_gt=('score_gt','max'),
            score_pop=('score_pop','max'),
            score_pc=('score_pc','max'))
      )

def make_icon_row(row):
    icons = []
    if row['causal_sign'] == 1:  icons.append('‡')
    elif row['causal_sign'] == -1: icons.append('¶')
    # if row['treated_any'] == 1: icons.append('📬')
    if row['outcome_any'] == 1: icons.append('🛒')
    return ''.join(icons)

agg['icons'] = agg.apply(make_icon_row, axis=1)

def topk_strings(df, score_col, k=TOPK):
    part = df.sort_values(score_col, ascending=False).head(k).copy()

    return [f"{r.icons} {r.item_name}".strip()
            for _, r in part.iterrows()]

col_gt  = topk_strings(agg, 'score_gt',  TOPK)
col_pop = topk_strings(agg, 'score_pop', TOPK)
col_pc  = topk_strings(agg, 'score_pc',  TOPK)

def pad(lst, k):
    return lst + ['']*(k-len(lst))
table = pd.DataFrame({
    'DLCE (ground-truth prop.)': pad(col_gt, TOPK),
    'DLCE (PropCare prop.)'    : pad(col_pop, TOPK),
    'DLCE (Popularity prop.)'  : pad(col_pc, TOPK),
})

from IPython.display import display
display(table)


,DLCE (ground-truth prop.),DLCE (PropCare prop.),DLCE (Popularity prop.)
0,CAN DOG FOOD (SKIPPY/PEDIGREE/,‡🛒 SEASONAL,CORN WHITE
1,WILD BIRD SEED,¶ PAPERBACK BEST SELLER,WILD BIRD SEED
2,🛒 SEASONAL,BREAD:PITA/POCKET BREADS,GRAHAM CRACKERS
3,‡🛒 SEASONAL,DOG TREATS (SOFT TREATS),‡🛒 SEASONAL
4,‡🛒 BABY SPRING WATERS,🛒 BEDDING ACCESSORIES,¶ VENDING SIZE/SNGL SERVE CRACKE
5,CHARCOAL,‡🛒 ANTI-ACIDS,PACKAGE DINNERS: OTHER
6,‡🛒 FRESH,PACKAGE DINNERS: OTHER,‡🛒 ANTI-ACIDS
7,SEASONAL CANDY BOX NON-CHOCOLA,GLASS BAKEWARE,DIPILATORIES
8,BREAD:PITA/POCKET BREADS,¶ VENDING SIZE/SNGL SERVE CRACKE,ASEPTIC PACK JUICE AND DRINKS
9,¶ APPLES GRANNY SMITH (BULK&BAG),‡🛒 BABY SPRING WATERS,¶ PREMIUM BREAD
